In [1]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np

In [2]:
from szp_funcs import print_df, path_to_data, day_types, to_double

In [3]:
month = 'apr'
nday = 21

In [4]:
bad_stat = ['Отпуск по уходу за ребенком', 'Отпуск по беременности и родам']

In [5]:
res = pd.read_excel(path_to_data + month + '.xlsx')

In [6]:
res['stv'] = res.apply(lambda row: to_double(row, 'stv'), axis=1)

In [26]:
def give_norm(row):
    if row['status'] == 'Увольнение':
        return nday
    return row['nday']

In [27]:
res['nday'] = res.apply(lambda row: give_norm(row), axis=1)

In [18]:
print_df(res[(res.inn == 7736110982) & (res.type == day_types[2])],'qwe')

In [7]:
res['stv'].fillna(1, inplace=True)
res['nday'].fillna(1, inplace=True)
res['wday'].fillna(res['nday'], inplace=True)

In [11]:
res['spis'] = res.apply(lambda row: row['stv'] * row['wday'] / nday if row['wday'] <= nday else row['stv'], axis=1)

In [12]:
res

,inn,snils,sum,job,day,type,stv,status,status_pref,wday,nday,spis
0,7734408766,001-879-913 57,135113.91,Преподаватель,1,Основное место работы,1.0,Работа,Работа,22.0,21.0,1.000000
1,7734408766,001-896-420 39,126320.51,Преподаватель,1,Основное место работы,1.0,Работа,Работа,23.0,21.0,1.000000
2,7734408766,002-050-383 67,68007.79,Преподаватель,1,Основное место работы,1.0,Работа,Работа,21.0,21.0,1.000000
3,7734408766,002-065-271 85,76705.72,Преподаватель,1,Основное место работы,1.0,Работа,Работа,21.0,21.0,1.000000
4,7734408766,002-845-525 25,125265.61,Преподаватель,1,Основное место работы,1.0,Работа,Работа,22.0,21.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
198966,7722069597,186-373-917 14,96600.00,Специалист по интернет-маркетингу,1,Основное место работы,1.0,Работа,Работа,21.0,21.0,1.000000
198967,7722069597,187-703-552 01,38146.67,Воспитатель,0,Основное место работы,1.0,Работа,Отпуск учебный оплачиваемый,8.0,21.0,0.380952
198968,7722069597,198-391-256 24,83680.00,Воспитатель,1,Основное место работы,1.0,Работа,Работа,21.0,21.0,1.000000
198969,7722069597,202-693-820 52,84515.24,Учитель,1,Основное место работы,1.0,Работа,Работа,21.0,21.0,1.000000


In [13]:
ppl = res[res.status.isin(bad_stat) == False][['inn', 'type', 'snils']].groupby(['inn', 'type']).nunique().unstack().fillna(0.0)
ppl.index.names = ['inn']
ppl.columns = [s[1] + ' головы' for s in ppl.columns]

In [14]:
ppl[ppl.index == 7736110982]

,Внешнее совместительство головы,Внутреннее совместительство головы,Основное место работы головы
inn,,,
7736110982,90.0,287.0,1424.0


In [15]:
stv = res[res.status.isin(bad_stat) == False][['inn', 'type', 'stv']].groupby(['inn', 'type']).sum().unstack().fillna(0.0)
stv.index.names = ['inn']
stv.columns = [s[1] + ' ставки' for s in stv.columns]


In [16]:
stv[stv.index == 7736110982]

,Внешнее совместительство ставки,Внутреннее совместительство ставки,Основное место работы ставки
inn,,,
7736110982,45.133333,101.057233,1475.267


In [17]:
spis = res[res.status.isin(bad_stat) == False][['inn', 'type', 'spis']].groupby(['inn', 'type']).sum().unstack().fillna(0.0)
spis.index.names = ['inn']
spis.columns = [s[1] + ' списоч' for s in spis.columns]

In [18]:
spis[spis.index == 7736110982]

,Внешнее совместительство списоч,Внутреннее совместительство списоч,Основное место работы списоч
inn,,,
7736110982,36.256984,94.969614,1339.077238


In [43]:
print_df(pd.merge(pd.merge(ppl, stv, on='inn'), spis, on='inn'), 'Сравнение количества людей ' + month)